In [28]:
import requests
import tqdm
import pandas as pd

# Get oral questions

In [142]:
48918 // 500

97

In [143]:
data = []
for page in tqdm.notebook.tqdm(range(98)):
    url = f'https://lda.data.parliament.uk/commonsoralquestions.csv?_pageSize=500&_page={page}'
    data.append(requests.get(url))

In [144]:
df.shape

(5000, 16)

In [145]:
dfs = [pd.read_csv(StringIO(d.text)) for d in data]
df = pd.concat(dfs)
df.shape


(48918, 17)

In [146]:
df

,uri,answer date,answer date time,answering body,commons question time > question type,location > pref label,question status,ballot number,date tabled,modified,question text,tabling member > label,tabling member > uri,tabling member > constituency > pref label,tabling member printed,uin,tabling member
0,http://data.parliament.uk/resources/1388433,2022-01-05,2022-01-05T00:00:00,Prime Minister,Substantive,House of Commons chamber,Tabled,5,2021-12-16,2021-12-16T14:51:06.915,If he will list his official engagements for W...,Biography information for Sir Robert Neill,http://data.parliament.uk/members/1601,Bromley and Chislehurst,Sir Robert Neill,904884.0,NaN
1,http://data.parliament.uk/resources/1388434,2022-01-05,2022-01-05T00:00:00,Prime Minister,Substantive,House of Commons chamber,Tabled,3,2021-12-16,2021-12-16T14:51:09.551,If he will list his official engagements for W...,Biography information for Sarah Owen,http://data.parliament.uk/members/4777,Luton North,Sarah Owen,904882.0,NaN
2,http://data.parliament.uk/resources/1388435,2022-01-05,2022-01-05T00:00:00,Prime Minister,Substantive,House of Commons chamber,Tabled,15,2021-12-16,2021-12-16T14:51:12.243,If he will list his official engagements for W...,Biography information for Simon Fell,http://data.parliament.uk/members/4744,Barrow and Furness,Simon Fell,904894.0,NaN
3,http://data.parliament.uk/resources/1388436,2022-01-05,2022-01-05T00:00:00,Prime Minister,Substantive,House of Commons chamber,Tabled,1,2021-12-16,2021-12-16T14:51:14.923,If he will list his official engagements for W...,Biography information for Robert Halfon,http://data.parliament.uk/members/3985,Harlow,Robert Halfon,904880.0,NaN
4,http://data.parliament.uk/resources/1388437,2022-01-05,2022-01-05T00:00:00,Prime Minister,Substantive,House of Commons chamber,Tabled,13,2021-12-16,2021-12-16T14:51:17.563,If he will list his official engagements for W...,Biography information for Andy McDonald,http://data.parliament.uk/members/4269,Middlesbrough,Andy McDonald,904892.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,http://data.parliament.uk/resources/99512,2013-09-03,2013-09-03T00:00:00,Foreign and Commonwealth Office,Substantive,House of Commons chamber,Tabled,21,2013-08-27,2013-09-02T22:52:12.010,What recent assessment he has made of the situ...,Biography information for Sheila Gilmore,http://data.parliament.uk/members/3965,Edinburgh East,"Gilmore, Sheila",900095.0,NaN
414,http://data.parliament.uk/resources/99599,2013-09-04,2013-09-04T00:00:00,Prime Minister,Substantive,House of Commons chamber,Tabled,13,2013-08-27,2013-08-30T00:06:41.607,If he will list his official engagements for W...,Biography information for Mr Jim Hood,http://data.parliament.uk/members/582,Lanark and Hamilton East,"Hood, Mr Jim",900022.0,NaN
415,http://data.parliament.uk/resources/99603,2013-09-04,2013-09-04T00:00:00,Cabinet Office,Topical,House of Commons chamber,Tabled,1,2013-08-27,2013-09-02T22:52:12.010,If he will make a statement on his departmenta...,Biography information for Glyn Davies,http://data.parliament.uk/members/4041,Montgomeryshire,"Davies, Glyn",900100.0,NaN
416,http://data.parliament.uk/resources/99614,2013-09-04,2013-09-04T00:00:00,Cabinet Office,Substantive,House of Commons chamber,Tabled,15,2013-08-27,2013-09-04T22:27:58.123,What steps he is taking to open up central Gov...,Biography information for Mr Barry Sheerman,http://data.parliament.uk/members/411,Huddersfield,"Sheerman, Mr Barry",900039.0,NaN


In [60]:
!ls

Download Parliamentary Data.ipynb
Process HMD data - sentence level.ipynb
Process HMD data.ipynb
Process Manifesto Data for Sentence Classification.ipynb
Process Manifesto Data.ipynb


In [147]:
df.to_csv('../data/questions/parl_questions.csv')

# Get MP data

In [148]:
mp_ids = df['tabling member > uri'].unique() ; len(mp_ids)

981

In [149]:
mp = mp_ids[5]; mp

'http://data.parliament.uk/members/4805'

In [150]:
' https://lda.data.parliament.uk/members.json'

' https://lda.data.parliament.uk/members.json'

In [151]:
mp_data = [requests.get(f"https://lda.data.parliament.uk/members/{mpid.split('/')[-1]}.json") for mpid in mp_ids]

In [152]:
mp_data[10].json()['result']['primaryTopic']

{'_about': 'http://data.parliament.uk/members/1585',
 'additionalName': {'_value': 'William'},
 'constituency': {'_about': 'http://data.parliament.uk/resources/147313',
  'label': {'_value': 'Wimbledon'}},
 'familyName': {'_value': 'Hammond'},
 'fullName': {'_value': 'Stephen Hammond'},
 'gender': {'_value': 'Male'},
 'givenName': {'_value': 'Stephen'},
 'isPrimaryTopicOf': 'http://eldaddp.azurewebsites.net/members/1585.json',
 'label': {'_value': 'Biography information for Stephen Hammond'},
 'party': {'_value': 'Conservative'},
 'twitter': {'_value': 'https://twitter.com/s_hammond'}}

In [153]:
def get_info(mp_info):
    try:
        res = mp_info.json()['result']['primaryTopic']
        gender = res.get('gender',{}).get('_value',None)
        party = res.get('party',{}).get('_value',None)
        uri = res.get('_about',None)
        return [gender,party,uri]
    except:
        return [None,None,None]


In [154]:
#[get_info(mp) for mp in mp_data]

In [155]:
df_mp = pd.DataFrame([get_info(mp) for mp in tqdm.notebook.tqdm(mp_data)],
                    columns=['gender','party','uri_mp'])

In [156]:
df_mp

,gender,party,uri_mp
0,Male,Conservative,http://data.parliament.uk/members/1601
1,Female,Labour,http://data.parliament.uk/members/4777
2,Male,Conservative,http://data.parliament.uk/members/4744
3,Male,Conservative,http://data.parliament.uk/members/3985
4,Male,Labour,http://data.parliament.uk/members/4269
...,...,...,...
976,Female,Labour,http://data.parliament.uk/members/3915
977,Male,Liberal Democrat,http://data.parliament.uk/members/3946
978,Female,Conservative,http://data.parliament.uk/members/36
979,Male,None,http://data.parliament.uk/members/3955


In [157]:
df_merged = df.merge(df_mp,right_on='uri_mp',left_on='tabling member > uri')

In [158]:
df_populo = requests.get('https://cdn.jsdelivr.net/gh/everypolitician/everypolitician-data@8382ff6bd31785d630950716ccb53eccfbe8485f/data/UK/Commons/ep-popolo-v1.0.json').json()

In [159]:
len(df_populo['persons'])

1347

In [166]:
df_populo['persons'][10]

{'birth_date': '1962-05-06',
 'contact_details': [{'type': 'email', 'value': 'info@tombrake.co.uk'},
  {'type': 'phone', 'value': '020 7219 0924'},
  {'type': 'twitter', 'value': 'thomasbrake'}],
 'email': 'info@tombrake.co.uk',
 'gender': 'male',
 'given_name': 'Thomas',
 'id': '01fbdbc4-1cf7-4da3-a91c-1df42f806716',
 'identifiers': [{'identifier': '25726', 'scheme': 'bbc_democracy_live'},
  {'identifier': '2268', 'scheme': 'current_hansard'},
  {'identifier': '151', 'scheme': 'datadotparl'},
  {'identifier': '25726', 'scheme': 'dods'},
  {'identifier': '10063', 'scheme': 'everypolitician_legacy'},
  {'identifier': '/m/025ktk', 'scheme': 'freebase'},
  {'identifier': 'mr-tom-brake', 'scheme': 'hansard'},
  {'identifier': '144', 'scheme': 'historichansard'},
  {'identifier': 'tom-brake', 'scheme': 'journalisted'},
  {'identifier': 'commons/tom-brake/151', 'scheme': 'parliamentdotuk'},
  {'identifier': 'uk.org.publicwhip/person/10063', 'scheme': 'parlparse'},
  {'identifier': '2268', 's

In [167]:
people = requests.get('https://raw.githubusercontent.com/mysociety/parlparse/master/members/people.json').json()

In [191]:
people['persons'][10001]

{'id': 'uk.org.publicwhip/person/22008',
 'identifiers': [{'identifier': '8733', 'scheme': 'historichansard_person_id'},
  {'identifier': 'mr-kenneth-lindsay', 'scheme': 'historichansard_url'}],
 'other_names': [{'family_name': 'Lindsay',
   'given_name': 'Kenneth',
   'honorific_prefix': 'Mr',
   'note': 'Main'}]}

In [192]:
dob = []
#persons = df_populo['persons']
for p in tqdm.notebook.tqdm(people['persons']):
    
    for m_id in p.get('identifiers',[]):
        if m_id['scheme'] == 'datadotparl_id':
            dob.append([p,
                         'http://data.parliament.uk/members/'+ m_id['identifier'].split('/')[-1]])
len(dob)

2430

In [193]:
dob[0]

[{'id': 'uk.org.publicwhip/person/10001',
  'identifiers': [{'identifier': '7', 'scheme': 'historichansard_person_id'},
   {'identifier': '172', 'scheme': 'datadotparl_id'},
   {'identifier': 2567, 'scheme': 'yournextmp'},
   {'identifier': '3572', 'scheme': 'pims_id'},
   {'identifier': 'Q153454', 'scheme': 'wikidata'}],
  'other_names': [{'family_name': 'Abbott',
    'given_name': 'Diane',
    'note': 'Main'}],
  'shortcuts': {'current_constituency': 'Hackney North and Stoke Newington',
   'current_party': 'Labour'}},
 'http://data.parliament.uk/members/172']

In [160]:
dob = []
persons = df_populo['persons']
for p in persons:
    for m_id in p['identifiers']:
        if m_id['scheme'] == 'parliamentdotuk':
            dob.append([p['birth_date'],
                         'http://data.parliament.uk/members/'+ m_id['identifier'].split('/')[-1]])


In [161]:
dob_df = pd.DataFrame(dob,columns=['dob','uri_dob'])

In [162]:
df_merged = df_merged.merge(dob_df,left_on='tabling member > uri',right_on='uri_dob')

In [163]:
df_merged.to_csv('../data/questions/parl_questions.csv')

In [ ]:
https://lda.data.parliament.uk/members.json/